# Bayesian analysis of emotion-mediated memory as a function of power

Here, I am going to integrate neural data into my mixed-effects logistic regression models. 

In [ ]:
import arviz as az
import bambi as bmb
import argparse
from os.path import join
import pandas as pd
from pandas.api.types import CategoricalDtype
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from Bayesian_model_utils import run_model, plot_res, print_latex_table, plot_predictions

In [ ]:
# Model 1: No interaction effect here: 

pow_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/final_NHB_power_data_revised.csv', memory_map=True)

# rename the electrodes to properly do the elec random effects 
pow_df.elec = pow_df.apply(lambda x: f'{x.subj}_{x.elec}', axis=1)

# De-mean the valence  
pow_df['valence'] = pow_df['valence'] - 0.5

# Set categorical orders 

cat_type = CategoricalDtype(categories=['right', 'left'], ordered=True)
pow_df.hemi = pow_df.hemi.astype(cat_type)

cat_type = CategoricalDtype(categories=['Amy', 'Hipp'], ordered=True)
pow_df.region = pow_df.region.astype(cat_type)

# Let's do separate models for band

for band in ['theta']:
        
    model_df = pow_df[pow_df.band==band]

    y = 'memory'
    X = ['arousal', 'valence', 'region', 'hemi', 'power'] 
    Intx = ['power:arousal', 
            'power:valence',
            'power:hemi',
            'power:region',
            'power:region:hemi']

    label = (f"{y}" + "_{}"*len(X)).format(*X) + f'_{band}' + 'no_intx'

#             'power:hemi:region:arousal',
#             'power:hemi:region:CV'
    rand_effect = ['subj', 'elec']
    categorical = ['hemi', 'region']

    # Drop nan data
    model_df = model_df.dropna(subset=X)

    run_model(model_df, y, X, Intx, rand_effect, rand_slopes=False, categorical=categorical,
             chains=4, cores=4, tune=500, draws=1000, label=label, save_model_res=True)

In [ ]:
# Model 2: Examine the effect of HFA power and word features on recall. 

pow_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/final_NHB_power_data_revised.csv', memory_map=True)

# rename the electrodes to properly do the elec random effects 
pow_df.elec = pow_df.apply(lambda x: f'{x.subj}_{x.elec}', axis=1)

# De-mean the valence  
pow_df['valence'] = pow_df['valence'] - 0.5

# Set categorical orders 

cat_type = CategoricalDtype(categories=['right', 'left'], ordered=True)
pow_df.hemi = pow_df.hemi.astype(cat_type)

cat_type = CategoricalDtype(categories=['Amy', 'Hipp'], ordered=True)
pow_df.region = pow_df.region.astype(cat_type)

# Let's do separate models for band

for band in ['hfa', 'theta']:
        
    model_df = pow_df[pow_df.band==band]

    y = 'memory'
    X = ['arousal', 'valence', 'region', 'hemi', 'power'] 
    Intx = ['power:arousal', 
            'power:valence',
            'power:hemi',
            'power:region',
            'power:region:hemi',
            'power:arousal:valence']

    label = (f"{y}" + "_{}"*len(X)).format(*X) + f'_{band}' + '_all_elecs' + 'no_soz'

#             'power:hemi:region:arousal',
#             'power:hemi:region:CV'
    rand_effect = ['subj', 'elec']
    categorical = ['hemi', 'region']

    # Drop nan data
    model_df = model_df.dropna(subset=X)

    run_model(model_df, y, X, Intx, rand_effect, rand_slopes=False, categorical=categorical,
             chains=4, cores=4, tune=500, draws=1000, label=label, save_model_res=True)

In [ ]:
# Model 3: Fullest model 

pow_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/final_NHB_power_data_revised.csv', memory_map=True)

# rename the electrodes to properly do the elec random effects 
pow_df.elec = pow_df.apply(lambda x: f'{x.subj}_{x.elec}', axis=1)

# De-mean the valence  
pow_df['valence'] = pow_df['valence'] - 0.5

# Set categorical orders 

cat_type = CategoricalDtype(categories=['right', 'left'], ordered=True)
pow_df.hemi = pow_df.hemi.astype(cat_type)

cat_type = CategoricalDtype(categories=['Amy', 'Hipp'], ordered=True)
pow_df.region = pow_df.region.astype(cat_type)

# Let's do separate models for band

for band in ['hfa', 'theta']:
        
    model_df = pow_df[pow_df.band==band]

    y = 'memory'
    X = ['arousal', 'valence', 'region', 'hemi', 'power'] 
    Intx = ['power:arousal', 
            'power:valence',
            'power:hemi',
            'power:region',
            'power:region:hemi',
            'power:valence:hemi',
            'power:valence:region', 
            'power:arousal:hemi',
            'power:arousal:region',
            'power:arousal:valence']

    label = (f"{y}" + "_{}"*len(X)).format(*X) + f'_{band}' + '_all_elecs' + 'no_soz'

#             'power:hemi:region:arousal',
#             'power:hemi:region:CV'
    rand_effect = ['subj', 'elec']
    categorical = ['hemi', 'region']

    # Drop nan data
    model_df = model_df.dropna(subset=X)

    run_model(model_df, y, X, Intx, rand_effect, rand_slopes=False, categorical=categorical,
             chains=4, cores=4, tune=500, draws=1000, label=label, save_model_res=True)

In [ ]:
# Model (Control): Examine the effect of HFA power and word features on recall, plus hipp AP position (sontinuous)

pow_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/final_NHB_power_data_revised.csv', memory_map=True)

# rename the electrodes to properly do the elec random effects 
pow_df.elec = pow_df.apply(lambda x: f'{x.subj}_{x.elec}', axis=1)

# De-mean the valence  
pow_df['valence'] = pow_df['valence'] - 0.5

# Set categorical orders 

cat_type = CategoricalDtype(categories=['right', 'left'], ordered=True)
pow_df.hemi = pow_df.hemi.astype(cat_type)

cat_type = CategoricalDtype(categories=['Amy', 'Hipp'], ordered=True)
pow_df.region = pow_df.region.astype(cat_type)


# Restrict this to gamma 
pow_df = pow_df[pow_df.band=='hfa']

# Restrict this to hippocampus
pow_df = pow_df[pow_df.region=='Hipp']

# Bin longitudinal position
APbins = [-50, -20, 10]
pow_df['longitudinal_bin' ] = np.nan
pow_df['longitudinal_bin'] = pd.cut(pow_df['y_coord'], 
                                            bins=APbins, 
                                            labels=['posterior', 'anterior'])

cat_type = CategoricalDtype(categories=['posterior', 'anterior'], ordered=True)
pow_df.longitudinal_bin = pow_df.longitudinal_bin.astype(cat_type)


pow_df = pow_df.dropna(subset=['arousal', 'valence', 'region', 'power', 'hemi', 'y_coord'])


y = 'memory'
X = ['arousal', 'valence', 'y_coord', 'power', 'hemi'] 

Intx = ['power:arousal', 
        'power:valence',
        'power:hemi',
        'power:y_coord',
        'power:y_coord:hemi',
        'power:arousal:valence',
        'power:arousal:y_coord', 
        'power:valence:y_coord']

rand_effect = ['subj', 'elec']
categorical = ['hemi']

# Drop nan data
pow_df = pow_df.dropna(subset=X)

band='hfa'
label = (f"{y}" + "_{}"*len(X)).format(*X) + f'_{band}' + '_all_elecs' + 'no_soz'

run_model(pow_df, y, X, Intx, rand_effect, rand_slopes=False, categorical=categorical,
         chains=4, cores=4, tune=1000, draws=1000, label=label, save_model_res=True)

In [ ]:
# Model (Control): Examine the effect of HFA power and word features on recall, plus hipp AP position (binned)

pow_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/final_NHB_power_data_revised.csv', memory_map=True)

# rename the electrodes to properly do the elec random effects 
pow_df.elec = pow_df.apply(lambda x: f'{x.subj}_{x.elec}', axis=1)

# De-mean the valence  
pow_df['valence'] = pow_df['valence'] - 0.5

# Set categorical orders 

cat_type = CategoricalDtype(categories=['right', 'left'], ordered=True)
pow_df.hemi = pow_df.hemi.astype(cat_type)

cat_type = CategoricalDtype(categories=['Amy', 'Hipp'], ordered=True)
pow_df.region = pow_df.region.astype(cat_type)


# Restrict this to gamma 
pow_df = pow_df[pow_df.band=='hfa']

# Restrict this to hippocampus
pow_df = pow_df[pow_df.region=='Hipp']

# Bin longitudinal position
APbins = [-50, -20, 10]
pow_df['longitudinal_bin' ] = np.nan
pow_df['longitudinal_bin'] = pd.cut(pow_df['y_coord'], 
                                            bins=APbins, 
                                            labels=['posterior', 'anterior'])

cat_type = CategoricalDtype(categories=['posterior', 'anterior'], ordered=True)
pow_df.longitudinal_bin = pow_df.longitudinal_bin.astype(cat_type)


pow_df = pow_df.dropna(subset=['arousal', 'valence', 'region', 'power', 'hemi', 'longitudinal_bin'])


y = 'memory'
X = ['arousal', 'valence', 'longitudinal_bin', 'power', 'hemi'] 

Intx = ['power:arousal', 
        'power:valence',
        'power:hemi',
        'power:longitudinal_bin',
        'power:longitudinal_bin:hemi',
        'power:arousal:valence',
        'power:arousal:longitudinal_bin', 
        'power:valence:longitudinal_bin']

rand_effect = ['subj', 'elec']
categorical = ['hemi', 'longitudinal_bin']

# Drop nan data
pow_df = pow_df.dropna(subset=X)

band='hfa'
label = (f"{y}" + "_{}"*len(X)).format(*X) + f'_{band}' + '_all_elecs' + 'no_soz'

run_model(pow_df, y, X, Intx, rand_effect, rand_slopes=False, categorical=categorical,
         chains=4, cores=4, tune=1000, draws=1000, label=label, save_model_res=True)

Below you will find our one and only Bayesian linear mixed-effects model

In [ ]:
# Model 5: effect of stimulation on post-pre HFA and also every other power band. 

# run separate models for stimulation 

stim_t_test_data = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/FR2_stim_allbands_diff_newfilt_600ms_50buff_hfa.csv')

# replace the non-stim data with the word 2 stim data only
non_stim_path = '/home1/salman.qasim/Salman_Project/FR_Emotion/FR2_nostim_allbands_diff_newfilt_600ms_50buff_stimOFf_only.csv'

# Set categorical orders 

cat_type = CategoricalDtype(categories=['MTL', 'Amy', 'Hipp'], ordered=True)
stim_t_test_data.stim_reg = stim_t_test_data.stim_reg.astype(cat_type)

cat_type = CategoricalDtype(categories=['MTL', 'Amy', 'Hipp'], ordered=True)
stim_t_test_data.rec_reg = stim_t_test_data.rec_reg.astype(cat_type)

cat_type = CategoricalDtype(categories=['MTL', 'Amy', 'Hipp'], ordered=True)
stim_t_test_data.rec_reg = stim_t_test_data.rec_reg.astype(cat_type)

cat_type = CategoricalDtype(categories=['right', 'left'], ordered=True)
stim_t_test_data.stim_hemi = stim_t_test_data.stim_hemi.astype(cat_type)

cat_type = CategoricalDtype(categories=['right', 'left'], ordered=True)
stim_t_test_data.rec_hemi = stim_t_test_data.rec_hemi.astype(cat_type)



for sr in ['MTL', 'Hipp']:
    for rr in ['Hipp']:
        for y in ['theta_diff', 'alpha_diff', 'beta_diff', 'hfa_diff']:
            model_df = stim_t_test_data[(stim_t_test_data.stim_reg==sr)]
            model_df = model_df[(model_df.rec_reg==rr)]

            X = ['stim', 'stim_hemi'] 
            Intx = ['stim',
                    'stim:stim_hemi']

            label = (f"{y}" + "_{}"*len(X)).format(*X)+f'stim_{sr}_rec{rr}'

            rand_effect = 'sub'
            categorical = X

            # Drop nan data
            model_df = model_df.dropna()

            model, results = run_model(model_df, y, X, Intx, rand_effect, rand_slopes=False, categorical=categorical,
                     chains=4, cores=4, tune=1000, draws=1500, target_accept=0.95, model_fam='gaussian', return_model=True, label = label)

